In [ ]:
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip -q install sentencepiece Xformers einops
# !pip -q install langchain

In [ ]:
import torch
import transformers
from transformers import GenerationConfig, pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import bitsandbytes as bnb

import torch
import transformers
from transformers import GenerationConfig, pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ehartford/samantha-mistral-7b")

model = AutoModelForCausalLM.from_pretrained("ehartford/samantha-mistral-7b",
                                              load_in_8bit=True,
                                              device_map='auto',
                                              torch_dtype=torch.float16,
                                              low_cpu_mem_usage=True,
                                              )

In [ ]:
tokenizer.eos_token_id, tokenizer.pad_token_id
tokenizer.pad_token_id = 0
pipe = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=1536,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)


In [ ]:
import json
import textwrap

system_prompt = "A chat between a curious user and an artificial intelligence assistant. \nThe assistant gives helpful, detailed, and polite answers to the user's questions."

addon_prompt = "Your name is Samantha."
# USER: What is 4x8?
# ASSISTANT:
def get_prompt(human_prompt):
    # prompt_template=f"{human_prompt}"
    prompt_template = f"{addon_prompt}\n{system_prompt}\n\nUSER: {human_prompt} \nASSISTANT: "
    return prompt_template    

print(get_prompt('What is the meaning of life?'))

def remove_human_text(text):
    return text.split('USER:', 1)[0]

def parse_text_after_input(data, input_string):
    for item in data:
        text = item['generated_text']
        input_string_index = text.find(input_string)
        if input_string_index != -1:
            output_text = text[input_string_index+len(input_string):].strip()
            output_text = parse_text(output_text)
            wrapped_text = textwrap.fill(output_text, width=100)
            print(wrapped_text)

def parse_text(data):
    for item in data:
        text = item['generated_text']
        assistant_text_index = text.find('ASSISTANT:')
        if assistant_text_index != -1:
            assistant_text = text[assistant_text_index+len('ASSISTANT:'):].strip()
            assistant_text = remove_human_text(assistant_text)
            wrapped_text = textwrap.fill(assistant_text, width=100)
            print(wrapped_text)
            # return assistant_text

data = [{'generated_text': '### Human: What is the capital of England? \n### Response: The capital city of England is London.'}]
parse_text(data)

In [ ]:
# A chat between a curious user and an artificial intelligence assistant. 
# The assistant gives helpful, detailed, and polite answers to the user's questions.
# Your name is Samantha. 

# USER: What is the meaning of life? 
# ASSISTANT:

In [ ]:
%%time 
prompt = 'What is your name?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

In [ ]:
%%time 
prompt = 'What can you help me with?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

In [ ]:
%%time 
prompt = 'What are the difference between Llamas, Alpacas and Vicunas?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)
